In [1]:
#Import necessary libraries
import numpy as np
import torch
from pkg_resources import packaging
import numpy as np
import matplotlib.pyplot as plt


print("Torch version:", torch.__version__)

from transformers import AutoImageProcessor, FlavaImageModel, FlavaModel, FlavaFeatureExtractor, AutoModelForCausalLM
from datasets import load_dataset




Torch version: 2.0.0+cu117


/home/IAIS/jraghu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#check for GPU
torch.cuda.is_available()

True

In [3]:
#load the MNIST dataset using the load_dataset package
dataset = load_dataset("gorar/A-MNIST")

Found cached dataset a-mnist (/home/IAIS/jraghu/.cache/huggingface/datasets/gorar___a-mnist/amnist/1.1.0/49d6e25269c73523fbcc8d636818270c5604ddfdd1568ccabdcb39dc4416e954)
100%|██████████| 2/2 [00:00<00:00, 160.06it/s]


In [4]:
#oad CIFAR10 dataset also
cifar10 = load_dataset("cifar10")

Found cached dataset cifar10 (/home/IAIS/jraghu/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)
100%|██████████| 2/2 [00:00<00:00, 199.81it/s]


In [5]:
image = dataset["test"]["image"][0]
print(np.shape(image))
target=(30,30)
image = image.resize(target)
print(np.shape(image))
#Since the MNIST dataset is 2 dimensional and FLAVA needs 3 channel input, converting the MNIST data into 3 channel dataset
# Expand dimensions to add a channel dimension
train_images = np.expand_dims(image, axis=-1)

# Duplicate the grayscale channel into three channels
train_images = np.repeat(train_images, 3, axis=-1)

print(np.shape(train_images))

(28, 28)
(30, 30)
(30, 30, 3)


In [6]:
#use this to convert into 3 channel
t = image.convert("RGB")
print(np.shape(t))

(30, 30, 3)


In [5]:
#Call the main pre-trained FLAVA model API's required for building upon the pre-trained model
flava = FlavaModel.from_pretrained("facebook/flava-full").cuda().eval()
fe = FlavaFeatureExtractor.from_pretrained("facebook/flava-full")
image_processor = AutoImageProcessor.from_pretrained("facebook/flava-full")
model = FlavaImageModel.from_pretrained("facebook/flava-full")#.cuda().eval()


`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.
/home/IAIS/jraghu/.local/lib/python3.9/site-packages/transformers/models/flava/feature_extraction_flava.py:28: FutureWarning: The class FlavaFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use FlavaImageProcessor instead.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Checking out Image processor class

Making the MNIST dataset smaller for testing purposes

In [40]:
train_small = dataset["train"]["image"]
train_small_label = dataset["train"]["label"]

In [41]:
feature_list = []
rgb_images = []
target=(30,30)

#Iterate throught he MNIST dataset to convert it into 3 channel and append them into a list
for sample, label in zip(train_small, train_small_label):
  sample =sample.resize(target)
  
  rgb_image = sample.convert('RGB')
  
  # Append the RGB image to the list
  rgb_images.append(rgb_image)
        

In [23]:
# Move the model to the GPU (if not already on the GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
initial_memory = torch.cuda.memory_allocated(device=device)
device

device(type='cuda')

In [12]:
#inputs = image_processor(rgb_images, return_tensors="pt").to("cuda")

OutOfMemoryError: CUDA out of memory. Tried to allocate 67.29 GiB (GPU 0; 31.74 GiB total capacity; 1.22 GiB already allocated; 28.07 GiB free; 1.36 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# with torch.no_grad():
#     outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state

# last_hidden_states = last_hidden_states[:,0,:]

In [19]:
#Call the feature extractor on these rgb_images and then get the image features on this using the ViT Base -16 transformer unit
with torch.no_grad():
  image_rgb = fe(rgb_images, return_tensors="pt")#.to("cuda")
  #image_features = flava.get_image_features(**image_rgb)[:, 0, :]

In [20]:
image_features = flava.get_image_features(**image_rgb)[:, 0, :]

ValueError: Input image size (224*224) doesn't match model (30*30).

In [42]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    930 MiB |   1633 MiB |   4648 GiB |   4648 GiB |
|       from large pool |    928 MiB |    934 MiB |   2307 GiB |   2306 GiB |
|       from small pool |      2 MiB |    700 MiB |   2341 GiB |   2341 GiB |
|---------------------------------------------------------------------------|
| Active memory         |    930 MiB |   1633 MiB |   4648 GiB |   4648 GiB |
|       from large pool |    928 MiB |    934 MiB |   2307 GiB |

In [43]:
# Define the batch size
batch_size = 12

features_in_Tensor = []
# Calculate the number of batches
num_batches = len(rgb_images) // batch_size

# Process the images in batches
for batch_idx in range(num_batches):
    # Get the start and end indices for the current batch
    start_idx = batch_idx * batch_size
    end_idx = start_idx + batch_size

    # Get the images for the current batch
    batch_images = rgb_images[start_idx:end_idx]

    # Process the images in the current batch
    processed_images = []
    with torch.no_grad():
        for image in batch_images:
            # Your processing logic here

            processed_images = fe(image, return_tensors="pt").to("cuda")
            image_features = flava.get_image_features(**processed_images)[:, 0, :]

            features_in_Tensor.append(image_features.detach().cpu().numpy())
    # Do something with the processed images
    # For example, perform further computations or analysis
    
    # Clear memory of the processed images if no longer needed
    torch.cuda.empty_cache()
    del processed_images, image_features


Since the FLAVA paper mentions that they use a L-BFGS based logistic regression classifier as a classifier head on top of the image encoder to use the mebeddings, I have implemented the same from the same source as mentioned in the paper and further ahead is the results.

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Converting the tensor into Numpy 
#features = features_in_Numpy.detach().numpy()
features_in_Numpy = np.squeeze(features_in_Tensor, axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_in_Numpy, train_small_label, test_size=0.2, random_state=42)

# Create a logistic regression model with L-BFGS optimization
logistic_model = LogisticRegression(solver='lbfgs', max_iter=1000)

# Train the logistic regression model
logistic_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9549583333333334


In [58]:
print(np.shape(X_train), np.shape(X_test), np.shape(y_train),np.shape(y_test))

(96000, 768) (24000, 768) (96000,) (24000,)


In [62]:
#Preprocessing the test dataset to covert in ndarray
test_small = dataset["test"]["image"]
test_small_label = dataset["test"]["label"]

image_unseen = []

# Convert each image in the list to a numpy array
for image in test_small:
    # Convert the PIL image to a numpy array
    img_array = np.array(image)

    
    # Append the array to the 'image_arrays' list
    image_unseen.append(img_array)

print(np.shape(image_unseen), np.shape(test_small_label))
image_array = np.array(image_unseen)
image_array_flattened = image_array.reshape(10000, -1)
np.shape(image_array_flattened)

(10000, 28, 28) (10000,)


(10000, 784)

In [63]:
#Testing on completely unseen MNIST data

# Make predictions on the unseen dataset
y_pred_unseen = logistic_model.predict(image_array_flattened)

# Calculate accuracy
accuracy = accuracy_score(test_small_label, y_pred_unseen)
print("Accuracy:", accuracy)

ValueError: X has 784 features, but LogisticRegression is expecting 768 features as input.

CIFAR-10 with FLAVA-- Doing the whole same procedure as done above for MNISt for CIFAR-10 dataset

In [6]:
image_cifar = cifar10["train"]["img"][0]
tt = image_cifar.convert("RGB")
print(np.shape(tt))

(32, 32, 3)


In [21]:
train_cifar = cifar10["train"]["img"]
train_label_cifar = cifar10["train"]["label"]

In [22]:
feature_list_cifar = []
rgb_images_cifar = []

for sample, label in zip(train_cifar, train_label_cifar):
  # Convert to RGB
  rgb_image = sample.convert('RGB')
      
  # Append the RGB image to the list
  rgb_images_cifar.append(rgb_image)

In [23]:
#BAtching the Cifar10 dataset to get image features in batches to enable the efficeint and complete use of the GPU

def batching(batch_size, rgb_images):
    """
    Batch_size-> is the number o batches you want to process the dataset in; rgb_images-> the dataset list in RGB form
    returns a list of features of lists in tensors
    """

    # Define the batch size
    batch_size = 10

    features_in_Tensor = []
    # Calculate the number of batches
    num_batches = len(rgb_images) // batch_size

    # Process the images in batches
    for batch_idx in range(num_batches):
        # Get the start and end indices for the current batch
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size

        # Get the images for the current batch
        batch_images = rgb_images[start_idx:end_idx]

        # Process the images in the current batch
        processed_images = []
        with torch.no_grad():
            for image in batch_images:
                # Your processing logic here

                processed_images = fe(image, return_tensors="pt").to("cuda")
                image_features = flava.get_image_features(**processed_images)[:, 0, :]

                features_in_Tensor.append(image_features.detach().cpu().numpy())
        # Do something with the processed images
        
        
        # Clear memory of the processed images if no longer needed
        torch.cuda.empty_cache()
        del processed_images, image_features

    return features_in_Tensor


#Calling the batching function with parameters
features_in_Tensor = batching(batch_size=10, rgb_images=rgb_images_cifar)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Restructing the features in 2D format
features_in_Numpy = np.squeeze(features_in_Tensor, axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_in_Numpy, train_label_cifar, test_size=0.2, random_state=42)

# Create a logistic regression model with L-BFGS optimization
logistic_model = LogisticRegression(solver='lbfgs', max_iter=1000)

# Train the logistic regression model
logistic_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9578


In [20]:
#Preprocessing the test dataset to covert in ndarray
test_cifar = cifar10["test"]["img"]
test_label_cifar = cifar10["test"]["label"]

image_unseen = []

# Convert each image in the list to a numpy array
for image in test_cifar:
    # Convert the PIL image to a numpy array
    img_array = np.array(image)

    
    # Append the array to the 'image_arrays' list
    image_unseen.append(img_array)

print(np.shape(image_unseen), np.shape(test_label_cifar))
image_array = np.array(image_unseen)
image_array_flattened = image_array.reshape(10000, -1)
print(np.shape(image_array_flattened))
image_array_reshaped = np.concatenate((image_array_flattened[:, :1024], image_array_flattened[:, 1024:2048], image_array_flattened[:, 2048:]), axis=1)
print(np.shape(image_array_reshaped))

(10000, 32, 32, 3) (10000,)
(10000, 3072)
(10000, 3072)


In [ ]:
#Testing on completely unseen CIFAR data


# Make predictions on the unseen dataset
y_pred_unseen = logistic_model.predict(image_array_flattened)

# Calculate accuracy
accuracy = accuracy_score(test_small_label, y_pred_unseen)
print("Accuracy:", accuracy)